### IMPORTS

In [ ]:
%matplotlib widget
import sys
import math
from pathlib import Path
from operator import itemgetter

import numpy as np
import scipy.signal
from matplotlib import pyplot as plt


### CONFIGURATION

In [ ]:
DATADIR = ...
DATASET = ...
LOAD_MASSES = ...
MIN_EVENT_LENGTH = 10
MAX_EVENT_LENGTH = 150
MASS_FOR_SELECTION = ...
MASS_OF_INTEREST = ...

### MAIN

#### Generate numpy array

In [ ]:
with open(DATADIR / DATASET) as f:
    available = [int(m) for m in f.readline()[:-1].split("\t")[1:]]
    masses = dict(zip(available, list(range(1,len(available) + 1))))
    
for mass in LOAD_MASSES:
    assert mass in masses, f"Mass {mass} not in the available masses:{', '.join(m for m in masses)}"

ar = np.loadtxt(DATADIR / DATASET, delimiter="\t", skiprows=1, usecols=[masses[m] for m in LOAD_MASSES])

print(f"Loadad array with {ar.shape[1]} columns and {ar.shape[0]} rows, with a size of {sys.getsizeof(ar)} bytes.")

mass_dict = dict(zip(LOAD_MASSES, range(len(LOAD_MASSES))))

#### Get all peaks

In [ ]:
events_by_mass = {}
for i in range(ar.shape[1]):
    peaks = scipy.signal.find_peaks(ar[:,i],width=(MIN_EVENT_LENGTH, MAX_EVENT_LENGTH))[0]
    if not peaks.size:
        print("Could,'t detect any event for mass", LOAD_MASSES[i])
    peak_widths = scipy.signal.peak_widths(ar[:,i], peaks, rel_height=1.0)
    left = peak_widths[2]
    right = peak_widths[3]
    events_by_mass[LOAD_MASSES[i]] = list(zip(left.astype(int),right.astype(int)))

In [ ]:
events_by_mass

In [ ]:
# GET THE INTEGRAL OF EVERY EVENT
sum_I = np.array([i for i in [np.sum(ar[side[0]:side[1], mass_dict[MASS_OF_INTEREST]],axis=0) for side in events_by_mass[MASS_FOR_SELECTION]]])
print(f"{sum_I.size} detected events.")

In [ ]:
sum_I

In [ ]:
DT = 13e-6  # 13 µs == 13e-6 s
FLOW = 0.5  # 30 µL/min == 0.5 µL/s
TE = 0.1  # 0.1 = 10 %
# 1 ppb == 1 ng/mL == 1 fg/µL
TFE = DT * FLOW / TE

X0 = 0
X1 = 1
event_mass_interest = (sum_I - X0) / X1 * DT * FLOW / TE

### VALIDATION

In [ ]:
icp = ...

In [ ]:
plt.figure()
plt.boxplot((icp, event_mass_interest[event_mass_interest<120]))
plt.gca().set_xticklabels(["SC-ICP-MS","CyTOF"])
plt.gca().set_ylabel("mass (fg)")
plt.show()